In [1]:
from app.services.pricecharting import scrape_pricecharting_sales, scrape_pricecharting_set_cards, scrape_pricecharting_sets

url = "https://www.pricecharting.com/game/pokemon-base-set/charizard-1st-edition-4#completed-auctions"

sales = scrape_pricecharting_sales(url)

print(len(sales))


315


In [2]:
from app.services.pricecharting import compute_comp_stats

compute_comp_stats(sales)

{'Ungraded': {'count': 29,
  'median': 5150.92,
  'mean': 5569.51,
  'min': 2662.54,
  'max': 9600.0,
  'latest_sale': 8500.0},
 'Grade 7': {'count': 30,
  'median': 13200.0,
  'mean': 13899.26,
  'min': 6659.68,
  'max': 22500.0,
  'latest_sale': 20100.0},
 'Grade 8': {'count': 29,
  'median': 16000.0,
  'mean': 19369.04,
  'min': 8694.39,
  'max': 36547.04,
  'latest_sale': 34999.0},
 'Grade 9': {'count': 24,
  'median': 30750.0,
  'mean': 42440.68,
  'min': 9182.89,
  'max': 161850.0,
  'latest_sale': 75000.0},
 'Grade 9.5': {'count': 13,
  'median': 57789.0,
  'mean': 55169.85,
  'min': 10100.0,
  'max': 130000.0,
  'latest_sale': 86077.0},
 'PSA 10': {'count': 9,
  'median': 194650.0,
  'mean': 209211.36,
  'min': 28701.5,
  'max': 499999.0,
  'latest_sale': 194650.0},
 'Grade 1': {'count': 30,
  'median': 4400.0,
  'mean': 4402.64,
  'min': 3150.0,
  'max': 6000.0,
  'latest_sale': 4938.0},
 'Grade 2': {'count': 30,
  'median': 3349.99,
  'mean': 3836.85,
  'min': 2100.0,
  'max'

In [3]:
sets = scrape_pricecharting_sets()
sets[:20]

[{'name': 'Pokemon 1999 Topps Movie',
  'url': 'https://www.pricecharting.com/console/pokemon-1999-topps-movie'},
 {'name': 'Pokemon 1999 Topps Movie Evolution',
  'url': 'https://www.pricecharting.com/console/pokemon-1999-topps-movie-evolution'},
 {'name': 'Pokemon 1999 Topps TV',
  'url': 'https://www.pricecharting.com/console/pokemon-1999-topps-tv'},
 {'name': 'Pokemon 2000 Topps Chrome',
  'url': 'https://www.pricecharting.com/console/pokemon-2000-topps-chrome'},
 {'name': 'Pokemon 2000 Topps TV',
  'url': 'https://www.pricecharting.com/console/pokemon-2000-topps-tv'},
 {'name': 'Pokemon 2020 Battle Academy',
  'url': 'https://www.pricecharting.com/console/pokemon-2020-battle-academy'},
 {'name': 'Pokemon Ancient Origins',
  'url': 'https://www.pricecharting.com/console/pokemon-ancient-origins'},
 {'name': 'Pokemon Aquapolis',
  'url': 'https://www.pricecharting.com/console/pokemon-aquapolis'},
 {'name': 'Pokemon Arceus',
  'url': 'https://www.pricecharting.com/console/pokemon-arce

In [4]:
from bs4 import BeautifulSoup
import requests

url = "https://www.pricecharting.com/console/pokemon-base-set"
soup = BeautifulSoup(requests.get(url).text, "html.parser")

cards = soup.select("a[href*='/game/pokemon']")
print(len(cards))

100


In [36]:
cards = scrape_pricecharting_set_cards("pokemon-aquapolis")
print(len(cards))  # should be the full set count
for card in cards:
    print(card["name"], card["url"])

688
Ampharos #1 https://www.pricecharting.com/game/pokemon-aquapolis/ampharos-1
Ampharos [Reverse Holo] #1 https://www.pricecharting.com/game/pokemon-aquapolis/ampharos-reverse-holo-1
Entei #10 https://www.pricecharting.com/game/pokemon-aquapolis/entei-10
Entei [Reverse Holo] #10 https://www.pricecharting.com/game/pokemon-aquapolis/entei-reverse-holo-10
Porygon #103a https://www.pricecharting.com/game/pokemon-aquapolis/porygon-103a
Porygon [Reverse Holo] #103a https://www.pricecharting.com/game/pokemon-aquapolis/porygon-reverse-holo-103a
Porygon #103b https://www.pricecharting.com/game/pokemon-aquapolis/porygon-103b
Porygon [Reverse Holo] #103b https://www.pricecharting.com/game/pokemon-aquapolis/porygon-reverse-holo-103b
Arcanine #2 https://www.pricecharting.com/game/pokemon-aquapolis/arcanine-2
Arcanine [Reverse Holo] #2 https://www.pricecharting.com/game/pokemon-aquapolis/arcanine-reverse-holo-2
Ariados #3 https://www.pricecharting.com/game/pokemon-aquapolis/ariados-3
Ariados [Rever

In [45]:
import pandas as pd
import re
import numpy as np

# Make a DataFrame from your list of dicts
df = pd.DataFrame(cards)


# Parser function to extract parts
def parse_card_name(full_name: str):
    """
    Given a full card name like:
      'Oddish [Reverse Holo] #1'
    returns a tuple: (base_name, number, variant)
    """
    # 1) extract the variant in brackets (if any)
    variant_match = re.search(r"\[(.*?)\]", full_name)
    variant = variant_match.group(1).strip() if variant_match else None

    # 2) remove the bracketed portion for now
    name_no_variant = re.sub(r"\s*\[.*?\]", "", full_name)

    # 3) extract number (everything after # up to first whitespace)
    number_match = re.search(r"#([^\s]+)", name_no_variant)
    number = number_match.group(1) if number_match else None

    # 4) remove the trailing "#<number>"
    name_no_number = re.sub(r"\s*#\d+$", "", name_no_variant).strip()

    # 5) what's left is the base name
    base_name = name_no_number.strip()

    return base_name, number, variant

# Apply the parser to each row
df[["base_name", "number", "variant"]] = df["name"].apply(
    lambda n: pd.Series(parse_card_name(n))
)

df = df.drop_duplicates(subset="id", keep="first")
# See the result
df.head()

,name,url,id,base_name,number,variant
0,Ampharos #1,https://www.pricecharting.com/game/pokemon-aqu...,842530,Ampharos,1,NaN
1,Ampharos [Reverse Holo] #1,https://www.pricecharting.com/game/pokemon-aqu...,2293892,Ampharos,1,Reverse Holo
2,Entei #10,https://www.pricecharting.com/game/pokemon-aqu...,842539,Entei,10,NaN
3,Entei [Reverse Holo] #10,https://www.pricecharting.com/game/pokemon-aqu...,2293901,Entei,10,Reverse Holo
4,Porygon #103a,https://www.pricecharting.com/game/pokemon-aqu...,842623,Porygon #103a,103a,NaN


In [46]:
df["variant"].value_counts(dropna=False)

variant
NaN             189
Reverse Holo    151
Box Topper        4
Name: count, dtype: int64

In [48]:
print(df["variant"].isna().sum())
df.loc[df["variant"].isna()].head()

189


,name,url,id,base_name,number,variant
0,Ampharos #1,https://www.pricecharting.com/game/pokemon-aqu...,842530,Ampharos,1,NaN
2,Entei #10,https://www.pricecharting.com/game/pokemon-aqu...,842539,Entei,10,NaN
4,Porygon #103a,https://www.pricecharting.com/game/pokemon-aqu...,842623,Porygon #103a,103a,NaN
6,Porygon #103b,https://www.pricecharting.com/game/pokemon-aqu...,5135011,Porygon #103b,103b,NaN
8,Arcanine #2,https://www.pricecharting.com/game/pokemon-aqu...,842531,Arcanine,2,NaN


In [51]:
matches = df[df["name"].str.contains("ampharos", case=False, na=False)]
print(matches)

                           name  \
0                   Ampharos #1   
1    Ampharos [Reverse Holo] #1   
309                Ampharos #H1   

                                                   url       id     base_name  \
0    https://www.pricecharting.com/game/pokemon-aqu...   842530      Ampharos   
1    https://www.pricecharting.com/game/pokemon-aqu...  2293892      Ampharos   
309  https://www.pricecharting.com/game/pokemon-aqu...   887937  Ampharos #H1   

    number       variant  
0        1           NaN  
1        1  Reverse Holo  
309     H1           NaN  
